## Stabilization P = 1.4

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_max = 1.4;
t_max = 5.0;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 3.5, tauP = 9.5, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [3]:
hoomd.run(1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 756141 / 856000 | TPS 14.0582 | ETA 01:58:23
Time 00:00:20 | Step 756294 / 856000 | TPS 15.2698 | ETA 01:48:49
Time 00:00:30 | Step 756467 / 856000 | TPS 17.2332 | ETA 01:36:15
Time 00:00:40 | Step 756651 / 856000 | TPS 18.3763 | ETA 01:30:06
Time 00:00:50 | Step 756836 / 856000 | TPS 18.4707 | ETA 01:29:28
Time 00:01:00 | Step 757014 / 856000 | TPS 17.7959 | ETA 01:32:42
Time 00:01:10 | Step 757177 / 856000 | TPS 16.2786 | ETA 01:41:10
Time 00:01:20 | Step 757347 / 856000 | TPS 16.9708 | ETA 01:36:53
Time 00:01:30 | Step 757525 / 856000 | TPS 17.7091 | ETA 01:32:40
Time 00:01:40 | Step 757710 / 856000 | TPS 18.3625 | ETA 01:29:12
Time 00:01:50 | Step 757886 / 856000 | TPS 17.5919 | ETA 01:32:57
Time 00:02:00 | Step 758060 / 8

In [4]:
npt.set_params(tau = 4.5)

In [ ]:
hoomd.run(1e5)

** starting run **
Time 01:35:03 | Step 856181 / 956000 | TPS 18.067 | ETA 01:32:04
Time 01:35:13 | Step 856364 / 956000 | TPS 18.1711 | ETA 01:31:23
Time 01:35:23 | Step 856546 / 956000 | TPS 18.0874 | ETA 01:31:38
Time 01:35:33 | Step 856727 / 956000 | TPS 18.0766 | ETA 01:31:31
Time 01:35:43 | Step 856910 / 956000 | TPS 18.2807 | ETA 01:30:20
Time 01:35:53 | Step 857094 / 956000 | TPS 18.319 | ETA 01:29:59
Time 01:36:03 | Step 857278 / 956000 | TPS 18.3369 | ETA 01:29:43
Time 01:36:13 | Step 857459 / 956000 | TPS 18.0128 | ETA 01:31:10
Time 01:36:23 | Step 857642 / 956000 | TPS 18.2484 | ETA 01:29:49
Time 01:36:33 | Step 857823 / 956000 | TPS 18.0677 | ETA 01:30:33
Time 01:36:43 | Step 858007 / 956000 | TPS 18.2968 | ETA 01:29:15
Time 01:36:53 | Step 858192 / 956000 | TPS 18.3161 | ETA 01:28:59
Time 01:37:03 | Step 858379 / 956000 | TPS 18.5512 | ETA 01:27:42
Time 01:37:14 | Step 858562 / 956000 | TPS 18.2203 | ETA 01:29:07
Time 01:37:24 | Step 858747 / 956000 | TPS 18.3654 | ETA 01